In [1]:
import polars as pl
import altair as alt
import matplotlib.pyplot as plt

In [2]:
data = pl.read_csv("./datasets/results/pop_compare.csv")
data

FileNotFoundError: No such file or directory (os error 2): ./datasets/results/pop_compare.csv

In [4]:
origin = pl.read_excel("./datasets/original/poi_original.xlsx")
origin["Population"]

FileNotFoundError: no workbook found at path './datasets/original/poi_original.xlsx'

In [ ]:
data = data.with_columns(((pl.col("Official Estimate") - pl.col("Dataset Estimate"))*100/pl.col("Official Estimate")).alias("percent off"))

In [ ]:
# Create a bar chart comparing Dataset Estimate and Official Estimate
plt.figure(figsize=(10, 6))
cities = data['DISTRICT']

# Plotting the estimates
plt.bar(cities, data['Dataset Estimate'], width=0.4, label='Dataset Estimate', align='center')
plt.bar(cities, data['Official Estimate'], width=0.4, label='Official Estimate', align='edge')

# Adding labels and title
plt.xlabel('District')
plt.ylabel('Population Estimate')
plt.title('Comparison of Dataset Estimate(Last updated 18 February 2023) vs. Official Estimate(02/2021)')
plt.xticks(rotation=90)
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
data.select(pl.col("percent off")).mean()

In [5]:
full = pl.read_excel("./datasets/temp/around_poi_with_population.xlsx")
full

FileNotFoundError: no workbook found at path './datasets/temp/around_poi_with_population.xlsx'

In [ ]:
import folium


m = folium.Map(location=(21.022672, 105.831461), zoom_start=14)

poi_id = 105
poi = full.filter(pl.col("poi_id").eq(poi_id)).to_dicts()

folium.Marker(
        location=[float(poi[0]["lat"]), float(poi[0]["lon"])],
        tooltip="Click me!",
        popup="",
        icon=folium.Icon(color="green"),
    ).add_to(m)

for place in poi:
    folium.Marker(
        location=[float(place["latitude"]), float(place["longitude"])],
        tooltip=place["title"],
        icon=folium.Icon(color="red"),
    ).add_to(m)
m

In [ ]:
origin

In [7]:
full.join(origin, left_on='poi_id', right_on='Unique Identifier')["poi_id", "population", "Population"]

NameError: name 'full' is not defined

In [5]:
data

COUNTRY,CITY,DISTRICT,TYPE,Dataset Estimate,Official Estimate
str,str,str,str,f64,f64
"""Vietnam""","""Hà Nội""","""Đống Đa""","""Quận""",478.032,401.7
"""Vietnam""","""Hà Nội""","""Đông Anh""","""Huyện""",427.819,374.9
"""Vietnam""","""Hà Nội""","""Hoàng Mai""","""Quận""",424.121,364.9
"""Vietnam""","""Hà Nội""","""Hai Bà Trưng""","""Quận""",369.003,315.9
"""Vietnam""","""Hà Nội""","""Chương Mỹ""","""Huyện""",364.014,309.6
…,…,…,…,…,…
"""Vietnam""","""Hà Nội""","""Thanh Oai""","""Huyện""",196.813,185.4
"""Vietnam""","""Hà Nội""","""Hoàn Kiếm""","""Quận""",182.718,155.9
"""Vietnam""","""Hà Nội""","""Đan Phượng""","""Huyện""",181.365,154.3


In [6]:
dataset_chart = alt.Chart(data).mark_bar().encode(
    x='DISTRICT:N',
    y='Dataset Estimate:Q'
)
official_chart = alt.Chart(data).mark_tick(color='red').encode(
    x='DISTRICT:N',
    y='Official Estimate:Q',
)
dataset_chart | official_chart

alt.HConcatChart(...)

In [7]:
df = pl.read_parquet("./datasets/raw/area_pois.parquet")
df.unique(subset=["id"])

id,lat,lon,name,categories
str,f64,f64,str,str
"""ChIJ5Z_ve0etNTERhOp99cjvNro""",21.056839,105.784211,"""American Medical Clinic, Vietn…","""hospital,doctor,health,point_o…"
"""ChIJTx7ITI-tNTERLdDy93Vt5o0""",20.996316,105.80988,"""Chè Cha Wang Pudding HongKong""","""cafe,food,point_of_interest,es…"
"""ChIJQ4YADF2rNTERoPNG4fxPXmw""",21.03194,105.799918,"""Bank Coffee Hà Nội""","""cafe,store,food,point_of_inter…"
"""ChIJGfQ5At9VNDERjwOZCljLw94""",21.070407,105.707126,"""Agribank ATM""","""atm,finance,point_of_interest,…"
"""ChIJRxm7LW-sNTERoRwz8pO9SUw""",20.99109,105.843043,"""Circus Cafe""","""cafe,food,point_of_interest,es…"
…,…,…,…,…
"""ChIJ6Xr3fmGsNTERjDCRDE-U5no""",20.991663,105.832165,"""Indovina bank - Lê Trọng Tấn""","""bank,point_of_interest,finance…"
"""ChIJ7TyIzKarNTERYZgtjX5ALOI""",21.04947,105.796719,"""D PIANO""","""school,point_of_interest,store…"
"""ChIJPTUeEgCpNTERBaaDb1D080g""",21.057753,105.898072,"""Lớp Tiếng Anh cô Hạnh- E Land …","""school,point_of_interest,estab…"


In [8]:
import folium


m = folium.Map(location=(21.022672, 105.831461), zoom_start=14)

places = df.unique(subset=["id"]).sample(n=100).to_dicts()

for place in places:
    folium.Marker(
        location=[float(place["lat"]), float(place["lon"])],
        tooltip=place["name"],
        icon=folium.Icon(color="red"),
    ).add_to(m)
m

In [14]:
df = pl.read_csv("./pop.csv")
df

area,source,population
str,str,f64
"""Hà Nội""","""meta""",8.2747e6
"""Hà Nội""","""ghs""",7.4322e6
"""Cao Bằng""","""meta""",513973.673537
"""Cao Bằng""","""ghs""",482588.60852
"""Đắk Lắk""","""meta""",2.0680e6
…,…,…
"""Cần Thơ""","""tctk""",1.2589e6
"""Hậu Giang""","""tctk""",728300.0
"""Sóc Trăng""","""tctk""",1.1988e6


In [36]:
# Create the visualization
chart = alt.Chart(df).mark_bar().encode(
    y=alt.Y('area', title="District"),
    x=alt.X('population', title='Population'),
    color=alt.Color('source', title="Source"),
    yOffset="source",
).properties(width=800, height=900)
chart

alt.Chart(...)

In [23]:
    df.pivot("source", index="area").select(
        (abs(((pl.col("tctk") - pl.col("ghs")) / pl.col("tctk")))*100).alias("ghs"),
        (abs(((pl.col("tctk") - pl.col("meta")) / pl.col("tctk")))*100).alias("meta"),
    ).describe()

statistic,ghs,meta
str,f64,f64
"""count""",64.0,64.0
"""null_count""",0.0,0.0
"""mean""",12.177615,9.597109
"""std""",12.916889,9.512348
"""min""",0.512046,0.075875
"""25%""",4.407771,4.63957
"""50%""",10.433251,7.06713
"""75%""",16.286552,11.857699
"""max""",91.757263,59.533221
